**1- User defined architecture of DNN**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

def dsigmoid(x):
  s=1/(1+np.exp(-x))
  ds=s*(1-s)  
  return ds

def tanh(x):
    t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    return t

def dtanh(x):
    t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    dt=1-t**2
    return dt

def relu(p):
    return np.maximum(0, p)

def drelu(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

leaky_slope=0.1
def leakyrelu(x):
  return np.maximum(leaky_slope*x,x)

def dleakyrelu(x):
  d=np.zeros_like(x)
  d[x<=0]=leaky_slope
  d[x>0]=1
  return d
  
def softmax(u):
    return np.exp(u) / np.sum(np.exp(u), axis=0, keepdims=True)

In [0]:
# Initializing the Weights and Biases
parameters = {}
def initialize_parameters(layer_dims):
    L = len(layer_dims)
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * (np.sqrt(2 / layer_dims[l - 1]))
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

In [0]:
# Computing the Cost
def compute_cost(activation):
    loss = - np.sum((Y_train * np.log(activation["A"+str(n+1)])), axis=0, keepdims=True)
    cost = np.sum(loss, axis=1) / m
    return cost


In [0]:
# Predictions
def predict(parameters, X_test):
    forward_prop(parameters, X_test, activation)
    predictions = np.round(activation["A"+str(n+1)])
    return predictions

In [29]:
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = mnist.load_data()

Y_tr_resh = Y_train_orig.reshape(60000, 1)
Y_te_resh = Y_test_orig.reshape(10000, 1)
Y_tr_T = to_categorical(Y_tr_resh, num_classes=10)
Y_te_T = to_categorical(Y_te_resh, num_classes=10)
Y_train = Y_tr_T.T
Y_test = Y_te_T.T

X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.
print(Y_train.shape)


(10, 60000)


In [6]:
lst = [] 
lst.append(X_train.shape[0])
m = X_train.shape[1]
n = int(input("Enter number of Hidden layers : ")) 
print("Enter the size of hidden layers ")
for i in range(0, n): 
    ele = int(input()) 
    lst.append(ele)

lst.append(10)     
print(lst) 

Enter number of Hidden layers : 3
Enter the size of hidden layers 
256
128
64
[784, 256, 128, 64, 10]


In [0]:
# Forward Propagation
outputs = {}
activation = {}
def forward_prop(parameters, X_train, activation, fn=0):
    outputs["Z" + str(1)] = np.dot(parameters["W1"], X_train) + parameters["b1"]
    if fn==0:
      activation["A" + str(1)] = relu(outputs["Z" + str(1)])
    elif fn==1:
      activation["A" + str(1)] = sigmoid(outputs["Z" + str(1)])
    elif fn==2:
      activation["A" + str(1)] = tanh(outputs["Z" + str(1)])
    elif fn==3:
      activation["A" + str(1)] = leakyrelu(outputs["Z" + str(1)])

    for l in range(2, n+1):
        outputs["Z" + str(l)] = np.dot(parameters["W" + str(l)], activation["A" + str(l - 1)]) + parameters["b" + str(l)]
        if fn==0:
          activation["A" + str(l)] = relu(outputs["Z" + str(l)])
        elif fn==1:
          activation["A" + str(l)] = sigmoid(outputs["Z" + str(l)])
        elif fn==2:
          activation["A" + str(l)] = tanh(outputs["Z" + str(l)])
        elif fn==3:
          activation["A" + str(l)] = leakyrelu(outputs["Z" + str(l)])
        #activation["A" + str(l)] = relu(outputs["Z" + str(l)])
    outputs["Z"+str(n+1)] = np.dot(parameters["W"+str(n+1)], activation["A"+str(n)]) + parameters["b"+str(n+1)]
    activation["A"+str(n+1)] = softmax(outputs["Z"+str(n+1)])
    return outputs, activation

In [0]:
grad_reg = {}
m = X_train.shape[1]
def grad_re(parameters, outputs, activation, fn=0):
    grad_reg["dZ"+str(n+1)] = (activation["A"+str(n+1)] - Y_train) / m
    for l in range(1, n+1):
        grad_reg["dA" + str(n+1 - l)] = np.dot(parameters["W" + str(n+1 - l + 1)].T, grad_reg["dZ" + str(n+1 - l + 1)])
        if fn==0:
          grad_reg["dZ" + str(n+1 - l)] = grad_reg["dA" + str(n+1 - l)] * drelu(outputs["Z" + str(n+1 - l)])
        elif fn==1:
          grad_reg["dZ" + str(n+1 - l)] = grad_reg["dA" + str(n+1 - l)] * dsigmoid(outputs["Z" + str(n+1 - l)])
        elif fn==2:
          grad_reg["dZ" + str(n+1 - l)] = grad_reg["dA" + str(n+1 - l)] * dtanh(outputs["Z" + str(n+1 - l)])
        elif fn==3:
          grad_reg["dZ" + str(n+1 - l)] = grad_reg["dA" + str(n+1 - l)] * dleakyrelu(outputs["Z" + str(n+1 - l)])
    grad_reg["dW1"] = np.dot(grad_reg["dZ1"], X_train.T)
    grad_reg["db1"] = np.sum(grad_reg["dZ1"], axis=1, keepdims=True)
    for l in range(2, n+2):
        grad_reg["dW" + str(l)] = np.dot(grad_reg["dZ" + str(l)], activation["A" + str(l - 1)].T)
        grad_reg["db" + str(l)] = np.sum(grad_reg["dZ" + str(l)], axis=1, keepdims=True)
    return parameters, outputs, activation, grad_reg
#grad_re(parameters, outputs, activation)
def learning(grad_reg, learning_rate=0.005):
    for i in range(1, n+2):
        parameters["W" + str(i)] = parameters["W" + str(i)] - (learning_rate * grad_reg["dW" + str(i)])
        parameters["b" + str(i)] = parameters["b" + str(i)] - (learning_rate * grad_reg["db" + str(i)])
    return parameters
#learning(parameters, grad_reg, learning_rate=0.005)

In [12]:
# Iterating over num_iterations
print_cost = True
costs = []
from sklearn.metrics import classification_report, confusion_matrix

def model(num_iterations, costs, activation, fn=0):
    initialize_parameters(lst)
    for l in range(0, num_iterations):
        forward_prop(parameters, X_train, activation, fn)
        cost = compute_cost(activation)
        grad_re(parameters, outputs, activation, fn)
        learning(grad_reg, learning_rate=0.02)
        if l % 100 == 0:
            costs.append(cost)
        if print_cost and l % 100 == 0:
          forward_prop(parameters, X_test, activation, fn)
          A=activation["A"+str(n+1)]
          predictions = np.argmax(A, axis=0)
          labels = np.argmax(Y_test, axis=0)

          #print(confusion_matrix(predictions, labels))
          print(classification_report(predictions, labels))
          print("Cost after iteration %i: %f" % (l, cost))
    return costs, parameters

num_iterations = int(input("Enter number of iterations : "))
fn = int(input("Which activation function 0-relu, 1-sigmoid, 2-tanh, 3-leaky relu: "))
model(num_iterations, costs, activation, fn)
forward_prop(parameters, X_test, activation, fn)
A=activation["A"+str(n+1)]
predictions = np.argmax(A, axis=0)
labels = np.argmax(Y_test, axis=0)

#print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

Enter number of iterations : 600
[[  0   0   1   0   1   0   0   5   0   1]
 [  6  12   7  13  11  34  20  63  35  15]
 [  9 178  23  99  28  38  18  36  14  27]
 [ 13  55  18  12  38  39  20  17  28  30]
 [ 17   0  44  15  45   6   2   4  25  27]
 [  0   0   0   0   0   0   0   0   0   0]
 [  2  10   4   9  11  11   1  50  14  13]
 [119 460 130  97 307 232  77 352 320 354]
 [  0   0   0   0   0   0   0   0   0   0]
 [814 420 805 765 541 532 820 501 538 542]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.01      0.06      0.02       216
           2       0.02      0.05      0.03       470
           3       0.01      0.04      0.02       270
           4       0.05      0.24      0.08       185
           5       0.00      0.00      0.00         0
           6       0.00      0.01      0.00       125
           7       0.34      0.14      0.20      2448
           8       0.00      0.00      0.00      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 893    0   69    6    8   44  106    7   27   17]
 [   1 1084   34   11   24   54   37   29  100   24]
 [  13   18  676   45    3   21   28   31   30    2]
 [  21   11  157  890    3  461   60    9  423   16]
 [   1    0   19    2  699   37   30    9   28  169]
 [   0    0    1    0    0    8    0    0    0    0]
 [  29    4   17    8   13  105  668   12   14    7]
 [   2    1   13   10   47   80    3  676   31   94]
 [   4   17    6   11    2   29    6    7  204    3]
 [  16    0   40   27  183   53   20  248  117  677]]
              precision    recall  f1-score   support

           0       0.91      0.76      0.83      1177
           1       0.96      0.78      0.86      1398
           2       0.66      0.78      0.71       867
           3       0.88      0.43      0.58      2051
           4       0.71      0.70      0.71       994
           5       0.01      0.89      0.02         9
           6       0.70      0.76      0.73       877
           7       0.66      0.71   

NameError: ignored

In [0]:
forward_prop(parameters, X_test, activation, fn)
A=activation["A"+str(n+1)]
predictions = np.argmax(A, axis=0)
labels = np.argmax(Y_test, axis=0)

#print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

**Data Augmentation**

In [0]:
def data_augmentation(augment_size=5000): 
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255.0
    x_test /= 255.0
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    # Create one hot array
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)
    image_generator = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.05, 
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=False,
        vertical_flip=False, 
        data_format="channels_last",
        zca_whitening=True)
    # fit data for zca whitening
    image_generator.fit(x_train, augment=True)
    
    randidx = np.random.randint(train_size, size=augment_size)
    x_augmented = x_train[randidx].copy()
    y_augmented = y_train[randidx].copy()
    x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                                batch_size=augment_size, shuffle=False).next()[0]
    # append augmented data to trainset
    x_train = np.concatenate((x_train, x_augmented))
    y_train = np.concatenate((y_train, y_augmented))
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    Y_train = y_train.T
    Y_test = y_test.T

    X_train = x_train.reshape(x_train.shape[0], -1).T
    X_test = x_test.reshape(x_test.shape[0], -1).T
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    return X_train, Y_train, X_test, Y_test
        

In [0]:
# Iterating over num_iterations
print_cost = True
costs = []

def model(num_iterations, costs, activation, fn=0):
    initialize_parameters(lst)
    for l in range(0, num_iterations):
        forward_prop(parameters, X_train, activation)
        cost = compute_cost(activation)
        grad_re(parameters, outputs, activation)
        learning(grad_reg, learning_rate=0.02)
        if l % 100 == 0:
            costs.append(cost)
        if print_cost and l % 100 == 0:
          forward_prop(parameters, X_test, activation)
          A=activation["A"+str(n+1)]
          predictions = np.argmax(A, axis=0)
          labels = np.argmax(Y_test, axis=0)

          #print(confusion_matrix(predictions, labels))
          print(classification_report(predictions, labels))
          '''
          c=0 
          y=Y_test.T
          predictions = predict(parameters, X_test)
          p=predictions.T
          for i in range(10000):
            if np.array_equal(p[i], y[i]):
              c=c+1
          print(c/10000)
          print("Cost after iteration %i: %f" % (l, cost))'''
    return costs, parameters
print(X_train.shape)
print(X_train.shape)


num_iterations = int(input("Enter number of iterations : "))
f = int(input("Data Augmentation 0 for No and 1 for Yes : "))
if f==1:
  X_train, Y_train, X_test, Y_test= data_augmentation()
  print(Y_train.shape)
model(num_iterations, costs, activation)
forward_prop(parameters, X_test, activation)
A=activation["A"+str(n+1)]
predictions = np.argmax(A, axis=0)
labels = np.argmax(Y_test, axis=0)
print(classification_report(predictions, labels))

# Visualising the Cost vs num_iterations
plt.plot(costs)
plt.xlabel('iterations')
plt.ylabel('cost')
plt.show()

(784, 60000)
(784, 60000)
Enter number of iterations : 800
Data Augmentation 0 for No and 1 for Yes : 1


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


(784, 65000)
(10, 65000)
(784, 10000)
(10, 10000)
(10, 65000)
              precision    recall  f1-score   support

           0       0.07      0.03      0.04      2344
           1       0.22      0.13      0.16      1937
           2       0.01      0.55      0.01        11
           3       0.00      0.25      0.01        12
           4       0.05      0.10      0.07       463
           5       0.02      0.03      0.02       446
           6       0.12      0.08      0.10      1414
           7       0.01      0.10      0.02        96
           8       0.00      0.05      0.00        42
           9       0.61      0.19      0.29      3235

    accuracy                           0.11     10000
   macro avg       0.11      0.15      0.07     10000
weighted avg       0.28      0.11      0.15     10000

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      1069
           1       0.96      0.88      0.92      1244
           2    

In [28]:
print(X_train.shape)
print(Y_train.shape)

(784, 65000)
(65000,)
